In [ ]:
# Use to get any item from your google drive
from google.colab import drive
drive.mount('/content/drive')

!tar -xf /content/drive/My\ Drive/490data.tar.gz -C ./data

Mounted at /content/drive
tar: /content/drive/My Drive/490data.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
# This is supposed to make sure if you edit .py files
# colab will pick up on it
%load_ext autoreload
%autoreload 2

In [ ]:
!if [[ ! -d PROJECT ]] && [[ ! -d PIGEON ]]; then git clone https://github.com/iggy2k/CSC490-Project-Winter-2025.git PROJECT; fi

Cloning into 'PROJECT'...
remote: Enumerating objects: 412, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 412 (delta 31), reused 46 (delta 19), pack-reused 352 (from 1)
Receiving objects: 100% (412/412), 25.13 MiB | 15.82 MiB/s, done.
Resolving deltas: 100% (201/201), done.


In [ ]:
import os
import sys

# Local
if os.path.isdir('PIGEON/'):
    PIGEON_DIR = 'PIGEON/'
# Git clone when colab
else:
    PIGEON_DIR = 'PROJECT/PIGEON/'

os.environ["PIGEON_DIR"] = PIGEON_DIR
sys.path.insert(1, PIGEON_DIR)

In [ ]:
import yaml
from tqdm.notebook import tqdm
import subprocess


with open("./PROJECT/environment.yaml") as file_handle:
    environment_data = yaml.safe_load(file_handle)

for dependency in tqdm(environment_data["dependencies"], total=len(environment_data["dependencies"])):
    if isinstance(dependency, dict):
      for lib in tqdm(dependency['pip'], total=len(dependency['pip'])):
        try:
            out = subprocess.check_output(f'pip install {lib.split("=")[0]}', shell=True)
        except subprocess.CalledProcessError as err:
            print(err)
    else:
      try:
          subprocess.check_output(f'pip install {dependency.split("=")[0]}', shell=True)
      except subprocess.CalledProcessError as err:
          print(err)
try:
  subprocess.check_output('pip install pycountry global_land_mask', shell=True)
except subprocess.CalledProcessError as err:
  print(err)

  0%|          | 0/26 [00:00<?, ?it/s]

Command 'pip install matplotlib-base' returned non-zero exit status 1.
Command 'pip install python' returned non-zero exit status 1.
Command 'pip install pytorch' returned non-zero exit status 1.


  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
from huggingface_hub import snapshot_download
import os

if not os.path.isdir('datasets/osv5m/images'):
    snapshot_download(repo_id="osv5m/osv5m", local_dir="datasets/osv5m", allow_patterns=[
        'images/train/00.zip',
        'images/test/00.zip',
        '*.csv'
        ], repo_type='dataset')

[autoreload of jupyter_client.manager failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/usr/local/lib/python3.11/dist-packages/IPython/extensions/autoreload.py", line 266, in update_function
    setattr(old, name, getattr(new, name))
ValueError: _laun

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

test.csv:   0%|          | 0.00/116M [00:00<?, ?B/s]

00.zip:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

00.zip:   0%|          | 0.00/2.52G [00:00<?, ?B/s]

In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Point

import geopandas as gpd
from geopandas import GeoDataFrame

import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import zipfile

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torchvision import datasets, transforms

from PIL import Image
import pandas as pd

from tqdm.notebook import tqdm

In [ ]:
for root, dirs, files in os.walk("datasets/osv5m"):
    for file in files:
        if file.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(root, file), 'r') as zip_ref:
                for member in tqdm(zip_ref.infolist(), desc=f'Extracting {os.path.join(root, file)}'):
                  zip_ref.extract(member, root)
            os.remove(os.path.join(root, file))

Extracting datasets/osv5m/images/train/00.zip:   0%|          | 0/50001 [00:00<?, ?it/s]

Extracting datasets/osv5m/images/test/00.zip:   0%|          | 0/50001 [00:00<?, ?it/s]

In [ ]:
import pycountry

# Debug
MAX_ITEMS = 1000

class ImageCoordinateDataset(Dataset):
    def __init__(self, csv_file, image_dirs, transform=None):
        self.data = []
        self.files = []
        self.images = []
        self.skipped = 0
        self.csv = csv_file
        self.transform = transform

        for image_dir in image_dirs:
          print('Reading', image_dir)

          self.files.extend([f"{image_dir}/{f}" for f in listdir(image_dir) if isfile(join(image_dir, f))])

          print(f'Found {len(self.files)} files.')

          if not os.path.isdir('datasets/osv5m/'):
            os.makedirs('datasets/osv5m/')
          if isfile(f"{csv_file}_filtered.csv"):
            self.df = pd.read_csv(f"{csv_file}_filtered.csv", index_col=False)
          else:
            self.df = pd.concat([chunk for chunk in tqdm(pd.read_csv(self.csv, chunksize=5000, usecols=['id', 'latitude', 'longitude', 'country'], index_col=False), desc='Loading data')])

          print(f'Found {len(self.df)} csv entries.')

          self.df['country'] = self.df['country'].apply(lambda x: pycountry.countries.get(alpha_2=x).name if pycountry.countries.get(alpha_2=x) else x)

          new = pd.DataFrame(columns=['id', 'latitude', 'longitude', 'country'])
          i = 0
          for full_path in tqdm(self.files, total=len(self.files), desc='Processing files'):
              image_name = str(Path(full_path).stem)

              try:
                row = self.df[self.df['id'] == int(image_name)].iloc[0]
              except:
                continue
              new.loc[i] = row
              lat = row['latitude']
              lon = row['longitude']

              # Remove mislaballed images (ocean pictures?)
              # if not globe.is_land(float(lat), float(lon)):
              #   self.skipped += 1
              #   continue
              self.data.append(np.array([str(full_path), float(lat), float(lon)]))
              i += 1

          self.df = new
        if MAX_ITEMS is not None:
          print(f'Keeping {MAX_ITEMS} items')
          self.df = self.df[:MAX_ITEMS]
          self.data = self.data[:MAX_ITEMS]
          self.files = self.files[:MAX_ITEMS]

        self.df.to_csv(f"{csv_file}_filtered.csv", index=False)

        print(f'Dataset ready, {len(self.files)} files.')
        print(f'Skipped {self.skipped} non-land files.')


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx][0]
        coordinates = (float(self.data[idx][1]), float(self.data[idx][2]))
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(coordinates, dtype=torch.float32), self.data[idx][0]


In [ ]:
# https://www.geeksforgeeks.org/how-to-normalize-images-in-pytorch/
# The mean and std of ImageNet are: mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)
normalize = transforms.Normalize(mean.tolist(), std.tolist())

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

BATCH_SIZE = 32

In [ ]:
train_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/train.csv', image_dirs=[
                                      'datasets/osv5m/images/train/00',
                                      ],\
                                 transform=transform)
val_dataset = ImageCoordinateDataset(csv_file='datasets/osv5m/test.csv', image_dirs=[
                                      'datasets/osv5m/images/test/00',
                                      ],\
                                 transform=transform)


train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

Reading datasets/osv5m/images/train/00
Found 50000 files.
Found 1000 csv entries.


Processing files: 100%|██████████| 50000/50000 [00:13<00:00, 3694.94it/s]


Keeping 1000 items
Dataset ready, 1000 files.
Skipped 0 non-land files.
Reading datasets/osv5m/images/test/00
Found 50000 files.
Found 1000 csv entries.


Processing files: 100%|██████████| 50000/50000 [00:13<00:00, 3776.90it/s]

Keeping 1000 items
Dataset ready, 1000 files.
Skipped 0 non-land files.


In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('using cuda')
else:
  device = torch.device("cpu")
  print('using cpu')

using cuda


In [ ]:
CLIP_MODEL = 'openai/clip-vit-base-patch32'

In [ ]:
!pip install -q git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-923zk3rp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-923zk3rp
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=d7d597a422aca071f3b2930739b7c41684c453f5350294dc98f88ca70ac9b8dc
  Stored in directory: /tmp/pip-ephem-wheel-cache-hjuz6a14/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip
  Attempting uninstall: clip
    Found existing installation: clip 0.2.0
    Uninstalling clip-0.2.0:
      Successfully uninstalled clip-0.2.0


In [ ]:
# from transformers import CLIPVisionModel, CLIPProcessor
# embed_model = CLIPVisionModel.from_pretrained(CLIP_MODEL)
# embed_processor = CLIPProcessor.from_pretrained(CLIP_MODEL)

embed_model, embed_processor = clip.load('ViT-B/16', device)

In [ ]:
!mkdir -p data/geocells/
# Political boundaries (admin0 = country)
![ ! -f data/geocells/admin_2.geojson ] && wget --show-progress -O data/geocells/admin_2.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM2.geojson
![ ! -f data/geocells/admin_1.geojson ] && wget --show-progress -O data/geocells/admin_1.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.geojson
![ ! -f data/geocells/countries.geojson ] && wget --show-progress -O data/geocells/countries.geojson -q https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM0.geojson

In [ ]:
!pip install -q geojson

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from geojson import dump
import os
from tqdm import tqdm

# https://stackoverflow.com/questions/56605238/how-to-use-tqdm-for-json-file-load-progress-bar
def hook(obj):
    value = obj.get("features")
    if not value:
        return obj
    pbar = tqdm(value)
    for _ in pbar:
        pbar.set_description(f"Loading")
    return obj


def simplify_geojson(path: str, tolerance: float):

    new_filename = f'{path}_compressed_{tolerance}.geojson'

    new_file = Path(new_filename)
    if new_file.is_file():
        print(f'{new_filename} alredy exists, skipping...')
        return None

    with open(path) as f:
        features = json.load(f, object_hook=hook)["features"]

    file_stats = os.stat(path)
    print(f'{path} Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

    geom = GeometryCollection([shape(feature["geometry"]).buffer(0) for feature in features])
    polys = []
    pre = 0
    post = 0
    for p in geom.geoms:
        poly = p
        if poly.geom_type == 'Polygon':
            x = len(poly.exterior.coords)
        elif poly.geom_type == 'MultiPolygon':
            x = sum([len(poly_item.exterior.coords) for poly_item in poly.geoms])
        pre += x

        poly = poly.simplify(tolerance=tolerance)
        if poly.geom_type == 'Polygon':
            a = len(poly.exterior.coords)
        elif poly.geom_type == 'MultiPolygon':
            a = sum([len(poly_item.exterior.coords) for poly_item in poly.geoms])
        post += a

        polys.append(poly)

    print(f'Simplified {pre} polygons to {post}')
    p = gpd.GeoSeries(polys)
    p.plot()
    plt.show()

    with open(new_filename, 'w') as f:
        dump(GeometryCollection(polys), f)

    file_stats = os.stat(f'{path}_compressed_{tolerance}.geojson')
    print(f'{new_filename} Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

    return new_filename

In [ ]:
SIMPLIFY_GEOJSON = True

if SIMPLIFY_GEOJSON:
    print('Simplifying geojson file for faster geocell gen...')

    new_admin1 = simplify_geojson('data/geocells/admin_1.geojson', 2.5)
    new_admin2 = simplify_geojson('data/geocells/admin_2.geojson', 2.5)
    new_countries = simplify_geojson('data/geocells/countries.geojson', 2.5)

    COUNTRY_PATH = new_admin1
    ADMIN_1_PATH = new_admin2
    ADMIN_2_PATH = new_countries

    print('New geojson files:', COUNTRY_PATH, ADMIN_1_PATH, ADMIN_2_PATH)

Simplifying geojson file for faster geocell gen...
data/geocells/admin_1.geojson_compressed_2.5.geojson alredy exists, skipping...
data/geocells/admin_2.geojson_compressed_2.5.geojson alredy exists, skipping...
data/geocells/countries.geojson_compressed_2.5.geojson alredy exists, skipping...
New geojson files: data/geocells/countries.geojson data/geocells/admin_1.geojson data/geocells/admin_2.geojson


In [ ]:
# TODO: pre-generate all this, add to the repo and add option to load instead
import sys
sys.path.append(f'{PIGEON_DIR}/dataset_creation/geocell')
sys.path.append(PIGEON_DIR)

from geocell_creation import *

GEOCELL_PATH = 'data/geocells_yfcc.csv'
df = train_dataset.df
geocells_file = Path(GEOCELL_PATH)
if not geocells_file.is_file():
    geocell_creator = GeocellCreator(df, GEOCELL_PATH)
    geocells = geocell_creator.generate()
else:
  print('Loading existing geocells')

Loading existing geocells


In [ ]:
# Use to load anything to your google drive
from google.colab import drive
drive.mount('/content/drive')
!tar -zcf 490data.tar.gz data/
!cp 490data.tar.gz /content/drive/My\ Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
^C


In [ ]:
!pip install -q huggingface-hub transformers pygeos
!pip install -q -U dill datasets

In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
import torchvision
from torchvision import transforms

import torch
import pandas as pd
from torch import nn, Tensor
from config import *
from preprocessing.geo_utils import *
from preprocessing.utils import *
GEOCELL_PATH = 'data/geocells_yfcc.csv'

#TODO: move over more stuff from PIGEON

class GeoLocationModel(nn.Module):
    def __init__(self):
        super(GeoLocationModel, self).__init__()

        self.panorama = False
        # self.hidden_size = embed_dim
        self.serving = False
        self.should_smooth_labels = False
        self.multi_task = False
        # self.heading = heading
        self.yfcc = None
        self.freeze_base = False
        self.hierarchical = False
        self.num_candidates = 5

        # Save variables
        self.base_model = embed_model

        from pprint import pprint
        pprint(vars(embed_model))

        self.processor = embed_processor

        # Setup
        self._set_hidden_size()
        geocell_path = GEOCELL_PATH
        self.lla_geocells = self.load_geocells(geocell_path)
        self.num_cells = self.lla_geocells.size(0)

        self.input_dim = self.hidden_size

        self.cell_layer = nn.Linear(self.input_dim, self.num_cells)
        self.softmax = nn.Softmax(dim=-1)

        # Freeze / load parameters
        # self._freeze_params()

        # Loss
        self.loss_fnc = nn.CrossEntropyLoss()



    def forward(self, x: Tensor, labels: Tensor, image_path: str):

        pixel_values = self.processor(x)
        embedding = self.encode_image(pixel_values)

        if self.mode == 'transformer':
            embedding = embedding.last_hidden_state
            embedding = torch.mean(embedding, dim=1)
        else:
            embedding = embedding.pooler_output
        output = embedding

        # Linear layer
        logits = self.cell_layer(output)
        # print('output', output)
        geocell_probs = self.softmax(logits)

        # Compute coordinate prediction
        geocell_preds = torch.argmax(geocell_probs, dim=-1)
        pred_LLH = torch.index_select(self.lla_geocells.data, 0, geocell_preds)

        # Get top 'num_candidates' geocell candidates
        geocell_topk = torch.topk(geocell_probs, self.num_candidates, dim=-1)

        # Soft labels based on distance
        distances = haversine_matrix(labels, self.lla_geocells.data.t())
        label_probs = smooth_labels(distances)

        loss_clf = self.loss_fnc(logits, label_probs)

        return {'pred': pred_LLH, 'loss': loss_clf}

    def load_geocells(self, path: str) -> Tensor:
        """Loads geocell centroids and converts them to ECEF format

        Args:
            path (str, optional): path to geocells. Defaults to GEOCELL_PATH.

        Returns:
            Tensor: ECEF geocell centroids
        """
        geo_df = pd.read_csv(path)
        lla_coords = torch.tensor(geo_df[['longitude', 'latitude']].values)
        lla_geocells = nn.parameter.Parameter(data=lla_coords, requires_grad=False)
        return lla_geocells

    def _set_hidden_size(self):
        """
        Determines the hidden size of the model
        """
        self.hidden_size = 512
        # if self.base_model is not None:
        #     try:
        #         self.hidden_size = self.base_model.hidden_size
        #         self.mode = 'transformer'

        #     except AttributeError:
        #         self.hidden_size = self.base_model.hidden_sizes[-1]
        #         self.mode = 'convnext'

    # def _freeze_params(self):
    #     """Freezes model parameters depending on mode
    #     """
    #     if self.base_model is not None:
    #         if self.freeze_base:
    #             for param in self.base_model.parameters():
    #                 param.requires_grad = False

    #         # Load parameters and freeze relevant parameters
    #         elif 'clip-vit' in self.base_model.config._name_or_path and not self.serving:
    #             head = CLIP_PRETRAINED_HEAD_YFCC if self.yfcc else CLIP_PRETRAINED_HEAD
    #             self.load_state(head)
    #             print(f'Initialized model parameters from model: {head}')
    #             for param in self.base_model.vision_model.encoder.layers[:-1].parameters():
    #                 param.requires_grad = False

model = GeoLocationModel()

# No need with accelerate
# model = model.to(device)

{'_backward_hooks': OrderedDict(),
 '_backward_pre_hooks': OrderedDict(),
 '_buffers': {},
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_is_full_backward_hook': None,
 '_load_state_dict_post_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': {'ln_final': LayerNorm((512,), eps=1e-05, elementwise_affine=True),
              'token_embedding': Embedding(49408, 512),
              'transformer': Transformer(
  (resblocks): Sequential(
    (0): ResidualAttentionBlock(
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
      )
      (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (mlp): Sequential(
        (c_fc): Linear(in_features=512, out_features=2048, bias=True)
        (gelu): QuickGELU

In [ ]:
# https://github.com/gastruc/osv5m/blob/main/models/losses.py
class HaversineLoss(nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()

    def forward(self, x, y):
        """
        Args:
            x: torch.Tensor Bx2
            y: torch.Tensor Bx2
        Returns:
            torch.Tensor: Haversine loss between x and y: torch.Tensor([B])
        Note:
            Haversine distance doesn't contain the 2 * 6371 constant.
        """
        lhs = torch.sin((x[:, 0] - y[:, 0]) / 2) ** 2
        rhs = (
            torch.cos(x[:, 0])
            * torch.cos(y[:, 0])
            * torch.sin((x[:, 1] - y[:, 1]) / 2) ** 2
        )
        a = lhs + rhs
        return torch.arctan2(torch.sqrt(a), torch.sqrt(1 - a))

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR

haversineLoss = HaversineLoss()
# mseLoss = nn.MSELoss()
# ceLoss = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
scheduler = ExponentialLR(optimizer, gamma=0.8)

In [ ]:
!pip install -q livelossplot

In [ ]:
from livelossplot import PlotLosses

In [ ]:
from ipywidgets import Output
OUTPUT_CONTEXT = Output()
display(OUTPUT_CONTEXT)

Output()

In [ ]:
model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)

# model.to(device)

In [ ]:
model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)

num_epochs = 10
avg_epoch_loss = []

plotlosses = PlotLosses(figsize=(10, 5))



for epoch in tqdm(range(num_epochs), total=num_epochs, desc='Epochs'):
  train_loss = []
  avg_loss = 0

  for images, coordinates, image_path in (pbar:= tqdm(train_dataloader, total=len(train_dataloader), desc=f'Epoch {epoch + 1} progress')):
    # detached = coordinates.detach().cpu()
    # labels = np.array([detached[0: ], detached[1: ]], dtype=float)
    # labels = torch.from_numpy(labels)
    if torch.cuda.is_available():
      images, coordinates = images.cuda(), coordinates.cuda()
      # labels = labels.cuda()

    optimizer.zero_grad()
    # print(images.get_device(),labels.get_device())
    outputs = model(images, coordinates, image_path)

    # print("LOSS:")
    # print(outputs['loss'].item())

    loss_haversine = haversineLoss(outputs['pred'], coordinates).mean(dim=-1)
    # loss_haversine.requires_grad = True

    # mse = mseLoss(outputs, coordinates)
    # cross_entropy = ceLoss(outputs, coordinates)

    # pbar.set_postfix_str(f'Loss: {loss_haversine:.5f}; \
    #                            \nOut: {outputs.mean(dim=0).tolist()}\
    #                            \nExpected: {coordinates.mean(dim=0).tolist()}')
    train_loss.append(loss_haversine.detach())

    with OUTPUT_CONTEXT:
      plotlosses.update({
          'Haversine': loss_haversine.item(),
          # 'MSE': mse.item(),
          'Cross Entropy': outputs['loss'].item()
      })
      plotlosses.send()

    accelerator.backward(outputs['loss'])
    # loss_haversine.backward()
    optimizer.step()

  scheduler.step()
  avg_loss = sum(train_loss) / len(train_dataloader)
  avg_epoch_loss.append(avg_loss)
  print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

Setting figsize to (10, 5)


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]


TypeError: CLIP.forward() missing 1 required positional argument: 'text'

In [ ]:
import matplotlib.pyplot as plt
avg_epoch_loss = torch.tensor(avg_epoch_loss, device =  'cpu')
plt.figure(figsize=(7, 5))
plt.plot(range(1, len(avg_epoch_loss) + 1), avg_epoch_loss, label='Loss', color='blue', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
model.eval()

In [ ]:
avg_val_haversine_loss = 0.0
batch_haversine = []

with torch.no_grad():  # Disable gradient calculation for validation
    for images, coordinates in tqdm(val_dataloader, total=len(val_dataloader), desc=f'Validating'):
        # Forward pass
        images, coordinates = images.cuda(), coordinates.cuda()
        outputs = model(images)

        loss_haversine = haversineLoss(outputs, coordinates).mean(dim=-1)
        batch_haversine.append(loss_haversine)
        avg_val_haversine_loss += loss_haversine

# Calculate average loss and haversine
avg_val_haversine_loss /= len(val_dataloader)

print(f'Validation Haversine Loss: {avg_val_haversine_loss:.4f}')

In [ ]:
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter

# Back from gpu
batch_haversine = torch.tensor(batch_haversine, device =  'cpu')

# Remove outliers
batch_haversine = median_filter(batch_haversine, size=100)

plt.figure(figsize=(5, 5))
plt.subplot(1, 1, 1)
plt.plot(range(1, len(batch_haversine) + 1), batch_haversine, label='Validation haversine', color='green', marker='o')
plt.xlabel('Batch Number')
plt.ylabel('haversine')
plt.title('Validation haversine loss per Batch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
from global_land_mask import globe
import matplotlib.pyplot as plt
import random

url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"

unnormalize = transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist())

def plot_predictions(model, dataloader, num_samples=5):
    model.eval()
    with torch.no_grad():
        for images, coordinates in dataloader:

            images = images.cuda()

            outputs = model(images)

            images = images.cpu()
            outputs = outputs.cpu()

            rand_index = random.sample(range(0, len(images) - 1), min(num_samples, len(images) - 1))

            for i in range(min(num_samples, len(images))):
                i = rand_index[i]

                pred_lat, pred_lon = outputs[i].cpu().numpy()
                true_lat, true_lon = coordinates[i].numpy()

                haver_err = haversineLoss(
                                      torch.tensor(np.array([[pred_lon, pred_lat]]), dtype=torch.float32).deg2rad(),
                                      torch.tensor(np.array([[true_lon, true_lat]]), dtype=torch.float32).deg2rad(),
                                      )
                # Display the image
                img = images[i]
                img = unnormalize(img).permute(1, 2, 0).numpy()
                img = np.clip(img, 0, 1)

                plt.imshow(img)
                plt.title(f'Pred: ({pred_lat:.4f}, {pred_lon:.4f})\nTrue: ({true_lat:.4f}, {true_lon:.4f})\n Haversine: {haver_err}')
                plt.axis('off')

                # World map for better understanding of how bad our prediction is
                geometry = [Point(pred_lon, pred_lat), Point(true_lon, true_lat)]
                geo_df = GeoDataFrame(geometry = geometry)
                world = gpd.read_file(url)
                geo_df.plot(ax=world.plot(color="lightgrey", figsize=(10, 6)), marker='x', c=['red', 'green'], markersize=50);

                plt.show()
            break

In [ ]:
plot_predictions(model, val_dataloader, num_samples=10)